In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
!pip install vit-pytorch
from vit_pytorch import ViT
from tqdm import tqdm  # Import tqdm
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import numpy as np
!pip install seaborn
import seaborn as sns
import matplotlib.pyplot as plt

In [9]:
train_root_dir="/global/D1/plantVillageApple_processed/train"
val_root_dir="/global/D1/plantVillageApple_processed/validation"
test_root_dir="/global/D1/plantVillageApple_processed/test"

In [10]:
# Define the hyperparameters
batch_size = 32
learning_rate = 1e-4
num_epochs = 60

In [11]:
####################################
# Training
####################################

trans={
    # Train uses data augmentation
    'train':
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.4762, 0.3054, 0.2368],
                             [0.3345, 0.2407, 0.2164])
    ]),
    # Validation does not use augmentation
    'valid':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.4762, 0.3054, 0.2368],
                             [0.3345, 0.2407, 0.2164])
    ]),
    
    # Test does not use augmentation
    'test':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.4762, 0.3054, 0.2368],
                             [0.3345, 0.2407, 0.2164])
    ]),
}

In [12]:
#Generators
training_dataset = ImageFolder(train_root_dir,transform=trans['train'])
validation_dataset = ImageFolder(val_root_dir,transform=trans['valid'])
test_dataset = ImageFolder(test_root_dir,transform=trans['test'])

train_dataloader = DataLoader(training_dataset,batch_size,shuffle=True) # ** unpacks a dictionary into keyword arguments
val_dataloader = DataLoader(validation_dataset,batch_size)
test_dataloader = DataLoader(test_dataset,batch_size)

print('Number of Training set images:{}'.format(len(training_dataset)))
print('Number of Validation set images:{}'.format(len(validation_dataset)))
print('Number of Test set images:{}'.format(len(test_dataset)))

Number of Training set images:1903
Number of Validation set images:3171
Number of Test set images:793


In [13]:
# Initialize the Vision Transformer model
model = ViT(
    image_size=224,
    patch_size=32,
    num_classes=len(training_dataset.classes),
    dim=768,  # Dimension of the model
    depth=12,  # Number of transformer layers
    heads=12,  # Number of attention heads
    mlp_dim=3072,  # Dimension of the MLP layers
    dropout=0.1
)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Set the device to use (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print("Device: ", device)
model.to(device)

Device:  cuda


ViT(
  (to_patch_embedding): Sequential(
    (0): Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=32, p2=32)
    (1): LayerNorm((3072,), eps=1e-05, elementwise_affine=True)
    (2): Linear(in_features=3072, out_features=768, bias=True)
    (3): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (dropout): Dropout(p=0.0, inplace=False)
  (transformer): Transformer(
    (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (layers): ModuleList(
      (0-11): 12 x ModuleList(
        (0): Attention(
          (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attend): Softmax(dim=-1)
          (dropout): Dropout(p=0.1, inplace=False)
          (to_qkv): Linear(in_features=768, out_features=2304, bias=False)
          (to_out): Sequential(
            (0): Linear(in_features=768, out_features=768, bias=True)
            (1): Dropout(p=0.1, inplace=False)
          )
        )
        (1): FeedForward(
          (net): Sequential(
          

## Training

In [14]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    # Use tqdm for a progress bar
    with tqdm(total=len(train_dataloader), desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as pbar:
        for i, (images, labels) in enumerate(train_dataloader):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            # Update the progress bar
            pbar.set_postfix(loss=running_loss / (i + 1))
            pbar.update(1)

    print(f'Training - Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_dataloader)}')

    # Validation
    model.eval()
    val_loss = 0.0
    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for images, labels in val_dataloader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            true_labels.extend(labels.cpu().numpy())
            predicted_labels.extend(predicted.cpu().numpy())

    print(f'Validation - Epoch {epoch+1}/{num_epochs}, Loss: {val_loss/len(val_dataloader)}')

    # Compute and print validation accuracy
    accuracy = accuracy_score(true_labels, predicted_labels)
    print(f'Validation - Epoch {epoch+1}/{num_epochs}, Accuracy: {accuracy * 100:.2f}%')

# Save the trained model
torch.save(model.state_dict(), 'vit_model_plantVillage_Apple.pth')
print("Training completed and modal saved to vit_model.pth")

Epoch 1/60: 100%|███████████████████████| 60/60 [13:37<00:00, 13.62s/batch, loss=1.38]


Training - Epoch 1/60, Loss: 1.382939566175143
Validation - Epoch 1/60, Loss: 0.9107335421815514
Validation - Epoch 1/60, Accuracy: 64.33%


Epoch 2/60: 100%|██████████████████████| 60/60 [16:47<00:00, 16.79s/batch, loss=0.801]


Training - Epoch 2/60, Loss: 0.8008904784917832
Validation - Epoch 2/60, Loss: 0.6387656521052122
Validation - Epoch 2/60, Accuracy: 74.61%


Epoch 3/60: 100%|██████████████████████| 60/60 [17:50<00:00, 17.84s/batch, loss=0.546]


...

Training - Epoch 58/60, Loss: 0.08106654474589353
Validation - Epoch 58/60, Loss: 0.14381776200403693
Validation - Epoch 58/60, Accuracy: 95.74%


Epoch 59/60: 100%|███████████████████████| 60/60 [01:16<00:00,  1.28s/batch, loss=0.0723]


Training - Epoch 59/60, Loss: 0.07227583142424313
Validation - Epoch 59/60, Loss: 0.15527325654940796
Validation - Epoch 59/60, Accuracy: 94.42%


Epoch 60/60: 100%|███████████████████████| 60/60 [01:19<00:00,  1.32s/batch, loss=0.0838]


Training - Epoch 60/60, Loss: 0.08383982765953987
Validation - Epoch 60/60, Loss: 0.10279260755167342
Validation - Epoch 60/60, Accuracy: 96.66%
Training completed and modal saved to vit_model.pth
